# Calculate evaluation Plate and Datasplit evaluation results
Evaluation results include confusion matrices, pr curves, precision, recall, f1-score, and accuracy.
This occurs for each plate across all splits.

In [1]:
import pathlib
from collections import defaultdict

import pandas as pd
from joblib import load
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_recall_curve,
    precision_score,
    recall_score,
)

## Find the root of the git repo on the host system

In [2]:
# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

## Define paths

### Input

In [ ]:
# Set data type for the model evaluation
data_type = "cleaned"

# Set path to data directory
data_path = pathlib.Path(f"{root_dir}/1.train_models/data")

# Set suffix for data files if using QC or cleaned data
if data_type == "cleaned":
    suffix = "_qc"
else:
    suffix = ""

evaldf = pd.read_parquet(f"{data_path}/nf1_model_pre_evaluation_results{suffix}.parquet")
model = load(f"{data_path}/trained_nf1_model{suffix}.joblib")
le = load(f"{data_path}/trained_nf1_model_label_encoder{suffix}.joblib")

In [4]:
evaldf

,probability_WT,datasplit,predicted_genotype,true_genotype,Metadata_Cytoplasm_Parent_Nuclei,Metadata_genotype,Metadata_Nuclei_Location_Center_Y,Metadata_gene_name,Metadata_Cytoplasm_Parent_Cells,Metadata_Cells_Location_Center_X,...,Metadata_Nuclei_Number_Object_Number,Metadata_Image_FileName_RFP,Metadata_WellRow,Metadata_Well,Metadata_Image_FileName_DAPI,Metadata_Site,Metadata_Cells_Number_Object_Number,Metadata_Image_FileName_CY5,Metadata_number_of_singlecells,Metadata_Nuclei_Location_Center_X
0,5.144222e-01,val,1,0,34,Null,800.158373,NF1,25,873.411928,...,34,F12_01_4_25_RFP_001_illumcorrect.tiff,F,F12,F12_01_1_25_DAPI_001_illumcorrect.tiff,25,25,F12_01_3_25_CY5_001_illumcorrect.tiff,647,878.813537
1,7.639919e-03,val,0,0,9,Null,211.538551,NF1,6,574.426157,...,9,F12_01_4_6_RFP_001_illumcorrect.tiff,F,F12,F12_01_1_6_DAPI_001_illumcorrect.tiff,6,6,F12_01_3_6_CY5_001_illumcorrect.tiff,647,577.821846
2,5.583088e-02,val,0,0,30,Null,645.026971,NF1,23,317.487509,...,30,D12_01_4_18_RFP_001_illumcorrect.tiff,D,D12,D12_01_1_18_DAPI_001_illumcorrect.tiff,18,23,D12_01_3_18_CY5_001_illumcorrect.tiff,623,347.342857
3,5.176248e-02,val,0,0,23,Null,462.045115,NF1,19,577.321406,...,23,B12_01_4_21_RFP_001_illumcorrect.tiff,B,B12,B12_01_1_21_DAPI_001_illumcorrect.tiff,21,19,B12_01_3_21_CY5_001_illumcorrect.tiff,531,603.479445
4,2.337870e-01,val,0,0,5,Null,188.089190,NF1,4,592.828338,...,5,E12_01_4_20_RFP_001_illumcorrect.tiff,E,E12,E12_01_1_20_DAPI_001_illumcorrect.tiff,20,4,E12_01_3_20_CY5_001_illumcorrect.tiff,607,561.152076
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16536855,3.844039e-09,shuffled_test,0,1,9,WT,261.259484,NF1,6,218.863998,...,9,E3_01_4_7_RFP_001_illumcorrect.tiff,E,E3,E3_01_1_7_DAPI_001_illumcorrect.tiff,7,6,E3_01_3_7_CY5_001_illumcorrect.tiff,120,209.181335
16536856,5.447732e-03,shuffled_test,0,1,9,WT,601.651914,NF1,5,419.307119,...,9,G3_01_4_17_RFP_001_illumcorrect.tiff,G,G3,G3_01_1_17_DAPI_001_illumcorrect.tiff,17,5,G3_01_3_17_CY5_001_illumcorrect.tiff,121,474.252990
16536857,1.223174e-06,shuffled_test,0,1,2,WT,202.774862,NF1,1,337.014648,...,2,G3_01_4_2_RFP_001_illumcorrect.tiff,G,G3,G3_01_1_2_DAPI_001_illumcorrect.tiff,2,1,G3_01_3_2_CY5_001_illumcorrect.tiff,121,356.811467
16536858,2.828662e-06,shuffled_test,0,1,6,WT,481.477537,NF1,4,831.565640,...,6,E3_01_4_10_RFP_001_illumcorrect.tiff,E,E3,E3_01_1_10_DAPI_001_illumcorrect.tiff,10,4,E3_01_3_10_CY5_001_illumcorrect.tiff,120,835.982252


### Outputs

In [5]:
eval_path = pathlib.Path("model_evaluation_data")
eval_path.mkdir(parents=True, exist_ok=True)

In [6]:
gene_column = "true_genotype"

def down_sample_by_genotype(_df):
    """
    Parameters
    ----------
    _df: Pandas Dataframe
        The data to be downsampled by the gene_column column.

    Returns
    -------
        The data down-sampled by genotype.
    """

    min_gene = _df[gene_column].value_counts().min()
    return (_df.groupby(gene_column, group_keys=False)
            .apply(lambda x: x.sample(n=min_gene, random_state=0))
            )

## Calculate evaluation metrics

In [7]:
# Define evaluation metric data
# The "metrics" include precision, recall, accuracy, and f1 scores
eval_mets = {
    met: defaultdict(list) for met in
    ("metrics", "precision_recall", "confusion_matrix")
}

# Labels of confusion matrices in dataframe
cm_true_labels = [
    le.classes_[0],
    le.classes_[0],
    le.classes_[1],
    le.classes_[1]
]

cm_pred_labels = [
    le.classes_[0],
    le.classes_[1],
    le.classes_[0],
    le.classes_[1]
]

def compute_metrics(_df, _plate, _split):
    """
    Parameters
    ----------
    _df: Pandas Dataframe
        Model data to be evaluated.

    _plate: String
        Name of the plate for storing the metrics

    _split: String
        Name of the data split for storing the metric
    """

    y_true = _df[gene_column]
    y_pred = _df["predicted_genotype"]
    y_proba = _df["probability_WT"]

    # Store metrics
    eval_mets["metrics"]["f1_score"].append(f1_score(y_true, y_pred))
    eval_mets["metrics"]["precision"].append(precision_score(y_true, y_pred))
    eval_mets["metrics"]["recall"].append(recall_score(y_true, y_pred))
    eval_mets["metrics"]["accuracy"].append(accuracy_score(y_true, y_pred))
    eval_mets["metrics"]["plate"].append(_plate)
    eval_mets["metrics"]["datasplit"].append(_split)

    # Store precision and recall data
    precision, recall, _ = precision_recall_curve(y_true, y_proba)
    pr_size = precision.shape[0]
    eval_mets["precision_recall"]["precision"].extend(precision.tolist())
    eval_mets["precision_recall"]["recall"].extend(recall.tolist())
    eval_mets["precision_recall"]["plate"].extend([_plate] * pr_size)
    eval_mets["precision_recall"]["datasplit"].extend([_split] * pr_size)

    # Store confusion matrices
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.flatten()
    cm_size = cm.shape[0]
    eval_mets["confusion_matrix"]["confusion_values"].extend(cm.tolist())
    eval_mets["confusion_matrix"]["true_genotype"].extend(cm_true_labels)
    eval_mets["confusion_matrix"]["predicted_genotype"].extend(cm_pred_labels)
    eval_mets["confusion_matrix"]["plate"].extend([_plate] * cm_size)
    eval_mets["confusion_matrix"]["datasplit"].extend([_split] * cm_size)

In [8]:
# Iterate through each data split
for split in evaldf["datasplit"].unique():

    # Calculate metrics for all plates
    df_temp = evaldf.loc[(evaldf["datasplit"] == split)].copy()
    compute_metrics(df_temp, "all_plates", split)

    # Calculate metrics for each plate
    for plate in evaldf["Metadata_Plate"].unique():
        df_temp = evaldf.loc[(evaldf["Metadata_Plate"] == plate) & (evaldf["datasplit"] == split)].copy()
        df_temp = down_sample_by_genotype(df_temp)
        compute_metrics(df_temp, plate, split)

### Save evaluation metrics and model coefficients for plotting

In [9]:
for met, met_data in eval_mets.items():
    pd.DataFrame(eval_mets[met]).to_parquet(f"{eval_path}/{met}_final_qc_model.parquet")

pd.DataFrame(
    {
        "feature_names": model.feature_names_in_,
        "feature_importances": model.coef_.reshape(-1)
    }
).to_parquet(f"{eval_path}/feature_importances_qc.parquet")